In [1]:
import pandas as pd
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from sklearn import model_selection
import statsmodels.api as sm

In [2]:
emlakverileri=pd.read_excel("makineveri.xlsx")
df=emlakverileri.copy()
df=df.dropna()
df.head()

,m² (Brüt),Oda Sayısı,Bina Yaşı,Balkon,Kullanım Durumu,Kimden,Fiyat Endeksi,Fiyat,Yer
0,140,2,5,1,1,1,1,820000,1
1,180,3,4,1,2,2,1,550000,1
2,180,3,0,1,0,1,1,550000,1
3,190,3,1,1,0,1,1,400000,1
4,110,3,5,1,0,1,2,235000,1


In [3]:
df

,m² (Brüt),Oda Sayısı,Bina Yaşı,Balkon,Kullanım Durumu,Kimden,Fiyat Endeksi,Fiyat,Yer
0,140,2,5,1,1,1,1,820000,1
1,180,3,4,1,2,2,1,550000,1
2,180,3,0,1,0,1,1,550000,1
3,190,3,1,1,0,1,1,400000,1
4,110,3,5,1,0,1,2,235000,1
...,...,...,...,...,...,...,...,...,...
95,190,3,1,1,0,2,1,410000,1
96,110,2,0,1,2,2,1,315000,1
97,110,2,0,1,2,2,1,287000,1
98,157,3,0,1,0,2,3,315000,0


In [4]:
df.columns

Index(['m² (Brüt)', 'Oda Sayısı', 'Bina Yaşı', 'Balkon', 'Kullanım Durumu',
       'Kimden', 'Fiyat Endeksi ', 'Fiyat ', 'Yer '],
      dtype='object')

In [5]:
df = df[["m² (Brüt)","Oda Sayısı","Bina Yaşı","Balkon","Kullanım Durumu","Kimden","Fiyat ","Yer "]]

In [6]:
df

,m² (Brüt),Oda Sayısı,Bina Yaşı,Balkon,Kullanım Durumu,Kimden,Fiyat,Yer
0,140,2,5,1,1,1,820000,1
1,180,3,4,1,2,2,550000,1
2,180,3,0,1,0,1,550000,1
3,190,3,1,1,0,1,400000,1
4,110,3,5,1,0,1,235000,1
...,...,...,...,...,...,...,...,...
95,190,3,1,1,0,2,410000,1
96,110,2,0,1,2,2,315000,1
97,110,2,0,1,2,2,287000,1
98,157,3,0,1,0,2,315000,0


In [7]:
X = df.drop(["Fiyat "], axis = 1)
y = df["Fiyat "]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 144)

In [9]:
# GridSearchCV

In [10]:
params = {"colsample_bytree":[0.4,0.5,0.6],
         "learning_rate":[0.01,0.02,0.09],
         "max_depth":[2,3,4,5,6],
         "n_estimators":[100,200,500,2000]}

In [11]:
xgb = XGBRegressor()

In [12]:
grid = GridSearchCV(xgb, params, cv = 10, n_jobs = -1, verbose = 2)

In [13]:
grid.fit(X_train, y_train)

Fitting 10 folds for each of 180 candidates, totalling 1800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   24.9s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:   55.2s
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 1800 out of 1800 | elapsed:  4.8min finished


GridSearchCV(cv=10,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, gamma=None,
                                    gpu_id=None, importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estimators=100, n_jobs=None,
                                    num_parallel_tree=None, random_state=None,
                                    reg_alpha=None, reg_lambda=None,
                                    scale_pos_weight=None, subsample=None,
                                    tree_method=None, validate_par

In [14]:
grid.best_params_

{'colsample_bytree': 0.6,
 'learning_rate': 0.02,
 'max_depth': 2,
 'n_estimators': 200}

In [15]:
xgb1 = XGBRegressor(colsample_bytree = 0.6, learning_rate = 0.02, max_depth = 2, n_estimators = 200)

In [16]:
model_xgb = xgb1.fit(X_train, y_train)

In [17]:
model_xgb.predict(X_test)[0:30]

array([203745.25, 361756.06, 464419.84, 349090.88, 562620.8 , 348388.66,
       274541.4 , 382902.28, 240173.06, 348388.66, 332415.12, 472571.78,
       539359.4 , 359641.62, 245445.69, 291403.16, 345767.38, 313888.7 ,
       410844.56, 349742.25, 354363.38, 414183.94, 464419.84, 374750.34,
       244794.16, 458083.28, 332834.53, 328447.1 , 341146.25, 512514.97],
      dtype=float32)

In [18]:
y_test[0:30]

79    170000
90    265000
28    510000
32    330000
18    780000
35    485000
17    132000
59    365000
99    170000
64    330000
51    310000
9     420000
30    440000
24    320000
75    198000
78    268000
65    395000
86    290000
92    475000
93    340000
50    299000
20    315000
73    700000
72    435000
71    360000
36    640000
98    315000
40    283000
83    350000
25    780000
Name: Fiyat , dtype: int64

In [19]:
np.sqrt(-1*(cross_val_score(model_xgb, X_test, y_test, cv=10, scoring='neg_mean_squared_error'))).mean()

140229.32358797826

In [ ]:
mo